## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.columns
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 200
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
count = application_df['CLASSIFICATION'].value_counts()
count_greater_than_1 = count[count > 1]

# Display the filtered counts
print(count_greater_than_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 1000
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts <= cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert INCOME_AMT to numeric, remove any non-numeric characters like $ or commas
application_df['INCOME_AMT'] = pd.to_numeric(application_df['INCOME_AMT'].replace({'\$': '', ',': ''}, regex=True), errors='coerce')

# Handle NaN values by filling with the median of the column
application_df['INCOME_AMT'].fillna(application_df['INCOME_AMT'].median(), inplace=True)

In [10]:
application_df.dtypes

APPLICATION_TYPE           object
AFFILIATION                object
CLASSIFICATION             object
USE_CASE                   object
ORGANIZATION               object
STATUS                      int64
INCOME_AMT                float64
SPECIAL_CONSIDERATIONS     object
ASK_AMT                     int64
IS_SUCCESSFUL               int64
dtype: object

In [11]:
# Feature Engineering
# Replace zero values in INCOME_AMT with a small number (e.g., 1)
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(0, 1)

application_df['ask_income_diff'] =application_df['ASK_AMT'] - application_df['INCOME_AMT']
application_df['ask_income_ratio'] = application_df['ASK_AMT'] / application_df['INCOME_AMT']

# Preview the new features
print(application_df[['INCOME_AMT', 'ASK_AMT', 'ask_income_diff', 'ask_income_ratio']])

       INCOME_AMT   ASK_AMT  ask_income_diff  ask_income_ratio
0             1.0      5000           4999.0            5000.0
1             1.0    108590         108589.0          108590.0
2             1.0      5000           4999.0            5000.0
3             1.0      6692           6691.0            6692.0
4             1.0    142590         142589.0          142590.0
...           ...       ...              ...               ...
34294         1.0      5000           4999.0            5000.0
34295         1.0      5000           4999.0            5000.0
34296         1.0      5000           4999.0            5000.0
34297         1.0      5000           4999.0            5000.0
34298         1.0  36500179       36500178.0        36500179.0

[34299 rows x 4 columns]


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df_encoded = pd.get_dummies(application_df, drop_first=True)
application_df_encoded.head()

,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,ask_income_diff,ask_income_ratio,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y
0,1,1.0,5000,1,4999.0,5000.0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,1,1.0,108590,1,108589.0,108590.0,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
2,1,1.0,5000,0,4999.0,5000.0,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
3,1,1.0,6692,1,6691.0,6692.0,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
4,1,1.0,142590,1,142589.0,142590.0,False,False,True,False,...,False,False,True,False,False,False,False,False,True,False


In [13]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])  # Features (exclude target column)
y = application_df_encoded['IS_SUCCESSFUL']  # Target
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model function that Keras Tuner will optimize
def build_model(hp):
    model = models.Sequential()
    
    # First hidden layer
    model.add(layers.Dense(
        units=hp.Int('units_1', min_value=64, max_value=128, step=32),
        activation=hp.Choice('activation_1', values=['relu', 'tanh']),
        input_dim=X_train_scaled.shape[1]
    ))
    
    # Second hidden layer
    model.add(layers.Dense(
        units=hp.Int('units_2', min_value=32, max_value=64, step=16),
        activation=hp.Choice('activation_2', values=['relu', 'tanh'])
    ))
    
    # Third hidden layer (optional)
    model.add(layers.Dense(
        units=hp.Int('units_3', min_value=16, max_value=32, step=8),
        activation=hp.Choice('activation_3', values=['relu', 'tanh'])
    ))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(
        hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    ), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Initialize the Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='keras_tuner',
    project_name='predictor'
)

# Start the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))


Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.7265306115150452

Best val_accuracy So Far: 0.7271137237548828
Total elapsed time: 00h 06m 11s


In [20]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4096      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 16)                1040      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 13409 (52.38 KB)
Trainable params: 13409 (52.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE